In [1]:
import numpy as np 
import numba 

In [68]:
_N = 8
N = _N**2
lattice = np.zeros(N, dtype=np.int64)
empty_mask = np.zeros(N, dtype=np.bool_)
pred_mask = np.zeros(N, dtype=np.bool_)
prey_mask = np.zeros(N, dtype=np.bool_)
# Fill
Nprey, Npred = int(N // 5), int(N // 5)
indices = np.random.choice(N, size=Nprey+Npred, replace=False)
prey_idxs = indices[:Nprey]
pred_idxs = indices[Nprey:]
prey_mask[prey_idxs] = True 
pred_mask[pred_idxs] = True 

In [75]:
lattice[prey_idxs] = -1 
lattice[pred_idxs] = 1
prey_indices = np.where(prey_mask)[0]
prey_indices

array([ 7,  8, 10, 12, 14, 25, 26, 30, 37, 44, 49, 54])

In [6]:
lattice = np.zeros(100000, dtype=np.int64)
lattice[1230:3450] = 1

In [7]:
%timeit np.where(lattice==1)[0]
%timeit np.argwhere(lattice==1).flatten()
%timeit np.flatnonzero(lattice==1)

61 µs ± 854 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
70.8 µs ± 408 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
64.3 µs ± 626 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
indices = np.arange(10000, dtype=np.int64)

In [15]:
%timeit np.flatnonzero(indices==100)
%timeit np.where(indices==100)[0]
%timeit np.argmax(indices==100)

10.8 µs ± 48.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
8.14 µs ± 19.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.09 µs ± 17.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
